# OctoFit DB Setup Notebook
Run each cell in order to: check MongoDB, run Django migrations, populate the database, and verify the data.

In [ ]:
import subprocess, sys

# Step 1: Check if MongoDB is running
result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
mongod_lines = [l for l in result.stdout.splitlines() if 'mongod' in l and 'grep' not in l]
if mongod_lines:
    print("MongoDB is RUNNING:")
    for l in mongod_lines:
        print(l)
else:
    print("MongoDB is NOT running — attempting to start...")
    start = subprocess.run(['sudo', 'systemctl', 'start', 'mongod'], capture_output=True, text=True)
    print(start.stdout, start.stderr)
    start2 = subprocess.run(['sudo', 'service', 'mongod', 'start'], capture_output=True, text=True)
    print(start2.stdout, start2.stderr)
    import time; time.sleep(2)
    result2 = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
    mongod_lines2 = [l for l in result2.stdout.splitlines() if 'mongod' in l and 'grep' not in l]
    if mongod_lines2:
        print("MongoDB started successfully.")
    else:
        print("WARNING: Could not confirm MongoDB is running. Proceeding anyway.")

In [ ]:
import subprocess

PYTHON = '/workspaces/skills-build-applications-w-copilot-agent-mode/octofit-tracker/backend/venv/bin/python'
MANAGE = '/workspaces/skills-build-applications-w-copilot-agent-mode/octofit-tracker/backend/manage.py'
CWD    = '/workspaces/skills-build-applications-w-copilot-agent-mode/octofit-tracker/backend'

# Step 2: makemigrations
print("=== makemigrations ===")
r = subprocess.run([PYTHON, MANAGE, 'makemigrations'], capture_output=True, text=True, cwd=CWD)
print(r.stdout)
if r.stderr: print("STDERR:", r.stderr)
print("Exit code:", r.returncode)

In [ ]:
# Step 3: migrate
print("=== migrate ===")
r = subprocess.run([PYTHON, MANAGE, 'migrate'], capture_output=True, text=True, cwd=CWD)
print(r.stdout)
if r.stderr: print("STDERR:", r.stderr)
print("Exit code:", r.returncode)

In [ ]:
# Step 4: populate_db
print("=== populate_db ===")
r = subprocess.run([PYTHON, MANAGE, 'populate_db'], capture_output=True, text=True, cwd=CWD)
print(r.stdout)
if r.stderr: print("STDERR:", r.stderr)
print("Exit code:", r.returncode)

In [ ]:
# Step 5: Verify collections with mongosh
print("=== Verify: collection names ===")
r = subprocess.run(
    ['mongosh', 'octofit_db', '--eval', 'db.getCollectionNames()'],
    capture_output=True, text=True
)
print(r.stdout)
if r.stderr: print("STDERR:", r.stderr)

print("\n=== Verify: users ===")
r = subprocess.run(
    ['mongosh', 'octofit_db', '--eval', 'db.users.find().pretty()'],
    capture_output=True, text=True
)
print(r.stdout)

print("\n=== Verify: teams ===")
r = subprocess.run(
    ['mongosh', 'octofit_db', '--eval', 'db.teams.find().pretty()'],
    capture_output=True, text=True
)
print(r.stdout)

print("\n=== Verify: activities ===")
r = subprocess.run(
    ['mongosh', 'octofit_db', '--eval', 'db.activities.find().pretty()'],
    capture_output=True, text=True
)
print(r.stdout)

print("\n=== Verify: leaderboard ===")
r = subprocess.run(
    ['mongosh', 'octofit_db', '--eval', 'db.leaderboard.find().pretty()'],
    capture_output=True, text=True
)
print(r.stdout)

print("\n=== Verify: workouts ===")
r = subprocess.run(
    ['mongosh', 'octofit_db', '--eval', 'db.workouts.find().pretty()'],
    capture_output=True, text=True
)
print(r.stdout)

In [ ]:
import subprocess

REPO = '/workspaces/skills-build-applications-w-copilot-agent-mode'

def run(cmd):
    r = subprocess.run(cmd, capture_output=True, text=True, cwd=REPO)
    print('$', ' '.join(cmd))
    if r.stdout: print(r.stdout)
    if r.stderr: print('STDERR:', r.stderr)
    print('exit:', r.returncode, '\n')
    return r.returncode

run(['git', 'add', '-A'])
run(['git', 'status'])
rc = run(['git', 'commit', '-m',
    'Add OctoFit Tracker Django backend with MongoDB, REST API, models, serializers, views, tests, admin, and populate_db management command'])
run(['git', 'push', 'origin', 'build-octofit-app'])